# Oppgave 4

In [93]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import plotly.express as px
from prettytable import PrettyTable

In [94]:

client_id = "aff7c34e-993d-4132-81bc-1df3e81d7868" 
end_date = datetime.today().date()
start_date = end_date - timedelta(days=365)  
station_id = "SN18700"

def visualiserer_data(client_id):

    sources_url = "https://frost.met.no/sources/v0.jsonld"
    try:
        response = requests.get(sources_url, auth=(client_id, ""), timeout=10)
        response.raise_for_status()
        sources_data = response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print("Response text:", response.text)
        return None
    except requests.exceptions.RequestException as err:
        print(f"Error fetching station data: {err}")
        return None
    
    if "data" not in sources_data:
        print("No station data returned from API.")
        return None
    
    stations = []
    for source in sources_data["data"]:
        if "name" in source and "id" in source:
            stations.append({
                "id": source["id"],
                "name": source["name"],
                "country": source.get("country", "")
            })
    
    stations_df = pd.DataFrame(stations)
    norway_stations = stations_df[stations_df["country"] == "Norge"]
    
    if norway_stations.empty:
        print("No Norwegian stations found.")
        return None
    
    return norway_stations

visualiserer_data(client_id)


,id,name,country
0,SN47230,ÅKRA UNGDOMSSKOLE,Norge
1,SN20952,STATFJORD C,Norge
2,SN23670,E16 RYFOSS,Norge
3,SN59450,STADLANDET,Norge
4,SN55000,LUSTER - ORNES,Norge
...,...,...,...
2102,SN97120,E6 BARTA,Norge
2103,SN61062,TOMREFJORD,Norge
2106,SN15270,JUVVASSHØE,Norge
2108,SN74780,NAMSVATN II,Norge


In [95]:
def get_station_id_by_name(client_id, search_name):
    stations = visualiserer_data(client_id)
    if stations is None or stations.empty:
        print("Fant ingen stasjoner.")
        return None

    # Søker etter stasjoner med navn som inneholder søketeksten
    mask = stations["name"].str.lower().str.contains(search_name.lower())
    matches = stations[mask]

    if matches.empty:
        print(f"Ingen stasjoner matchet {search_name}")
        return None

    if len(matches) == 1:
        station_id = matches.iloc[0]["id"]
        print(f"Fant stasjon: {matches.iloc[0]['name']} → ID: {station_id}")
        
    else:
        print("Flere mulige stasjoner funnet:")
        print(matches[["id", "name"]])
        # Velger første som default så noe blir gjort
        station_id = matches.iloc[0]["id"]
        print(f"Velger den første: {station_id}")

get_station_id_by_name(client_id, "Oslo - Lysaker")

Fant stasjon: OSLO - LYSAKER → ID: SN18970


In [96]:
def fetch_and_analyze_weather(client_id, station_id, start_date, end_date):
    if not start_date:
        end_date = date.today()
        start_date = end_date - timedelta(days=365)
    elif not end_date:
        end_date = date.today()

    # Elementer du vil analysere (id, navn, enhet)
    elements = [
        ("mean(air_temperature P1D)", "Temperatur", "°C"),
        ("mean(relative_humidity P1D)", "Fuktighet", "%"),
        ("sum(precipitation_amount P1D)", "Nedbør", "mm"),
        ("mean(wind_speed P1D)", "Vindhastighet", "m/s"),
    ]

    results = []

    for element_id, name, unit in elements:
        # Henter data
        url = "https://frost.met.no/observations/v0.jsonld"
        params = {
            "sources": station_id,
            "elements": element_id,
            "referencetime": f"{start_date}/{end_date}"
        }

        try:
            response = requests.get(url, params=params, auth=(client_id, ""), timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Feil ved henting av {name}: {e}")
            results.append((name, unit, "Feil", "Feil", "Feil"))
            continue

        data = response.json()
        values = []

        # Trekker ut verdier
        for obs in data.get("data", []):
            for el in obs.get("observations", []):
                if el["elementId"] == element_id and "value" in el:
                    values.append(el["value"])

        if values:
            mean_val = round(np.mean(values), 2)
            median_val = round(np.median(values), 2)
            std_val = round(np.std(values), 2)
        else:
            mean_val = median_val = std_val = "Ingen data"

        results.append((name, unit, mean_val, median_val, std_val))

    # Lager tabell
    table = PrettyTable()
    table.field_names = ["Element", "Enhet", "Gjennomsnitt", "Median", "Std.avvik"]
    for row in results:
        table.add_row(row)

    print(table)

fetch_and_analyze_weather(client_id, station_id, start_date, end_date)

+---------------+-------+--------------+--------+-----------+
|    Element    | Enhet | Gjennomsnitt | Median | Std.avvik |
+---------------+-------+--------------+--------+-----------+
|   Temperatur  |   °C  |     8.27     |  8.25  |    7.48   |
|   Fuktighet   |   %   |    77.84     |  79.0  |   15.63   |
|     Nedbør    |   mm  |     2.5      |  0.0   |    5.56   |
| Vindhastighet |  m/s  |     2.42     |  2.3   |    1.0    |
+---------------+-------+--------------+--------+-----------+


In [97]:
def fetch_weather_data(client_id, station_id, start_date, end_date, elements):
    url = "https://frost.met.no/observations/v0.jsonld"
    params = {
        "sources": station_id,
        "elements": ",".join(elements),
        "referencetime": f"{start_date}/{end_date}"
    }

    try:
        response = requests.get(url, params=params, auth=(client_id, ""), timeout=10)
        response.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error: {http_err}")
        print("Response text:", response.text)
        return None
    except requests.exceptions.RequestException as err:
        print(f"Error fetching data: {err}")
        return None

    return response.json()


In [98]:
def parse_weather_data(data, element_id):
    if not data or "data" not in data:
        print("No data returned from API.")
        return pd.DataFrame()

    observations = []
    for obs in data["data"]:
        for element in obs["observations"]:
            if element["elementId"] == element_id:
                observations.append({
                    "date": obs["referenceTime"],
                    "value": element["value"]
                })

    df = pd.DataFrame(observations)
    if df.empty:
        print("No data available for this element.")
        return df

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")
    return df


In [99]:
def plot_weather_data(df, element_name, station_name, start_date, end_date, unit=""):
    if df.empty:
        return

    fig = px.line(
        df,
        x="date",
        y="value",
        title=f"{element_name} – {station_name} ({start_date} til {end_date})",
        labels={"date": "Dato", "value": f"{element_name} ({unit})"},
        template="plotly_white"
    )

    fig.update_traces(
        mode="lines",
        hovertemplate="Dato: %{x|%Y-%m-%d}<br>Verdi: %{y:.1f} {unit}"
    )
    fig.update_layout(
        xaxis_title="Dato",
        yaxis_title=f"{element_name} ({unit})",
        hovermode="x unified",
        showlegend=False
    )

    fig.show()


In [100]:

element_id = "mean(air_temperature P1D)"
element_name = "Gjennomsnittlig temperatur"
unit = "C"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)


In [101]:
element_id = "mean(relative_humidity P1D)"
element_name = "Relativ fuktighet"
unit = "%"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)

In [102]:
element_id = "mean(wind_speed P1D)"
element_name = "Vindhastighet"
unit = "m/s"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)

In [103]:
element_id = "sum(precipitation_amount P1D)"
element_name = "Nedbør"
unit = "mm"

def plot_weather_Barplot(df, element_name, station_name, start_date, end_date, unit=""):
    if df.empty:
        print("Ingen data å vise.")
        return

    fig = px.bar(
        df,
        x="date",
        y="value",
        title=f"{element_name} - {station_name} ({start_date} til {end_date})",
        labels={"date": "Dato", "value": f"{element_name} ({unit})"},
        template="plotly_white"
    )

    fig.update_traces(
        hovertemplate="Dato: %{x|%Y-%m-%d}<br>Verdi: %{y:.1f} {unit}"
    )
    fig.update_layout(
        xaxis_title="Dato",
        yaxis_title=f"{element_name} ({unit})",
        hovermode="x unified",
        showlegend=False
    )

    fig.show()

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_Barplot(df, element_name, "Oslo - Blindern", start_date, end_date, unit)